In [1]:
!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 65.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 29.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not curre

In [2]:
import torch
from trl import SFTTrainer
from peft import LoraConfig
from datasets import load_dataset
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, pipeline)

In [3]:
import pandas as pd
from datasets import Dataset, DatasetDict

# Load the Excel file
file_path = '/content/FAQ_list_PAN_TAN_All.xlsx'
df = pd.read_excel(file_path)

# Initialize an empty DataFrame to store the cleaned data
cleaned_data = []

# Iterate over the rows in the DataFrame
for index, row in df.iterrows():
    questions = row['Question'].split('\n')  # Split the questions if there are multiple in one cell
    answer = row['Answer'].strip()  # Get the corresponding answer

    for question in questions:
        question = question.strip()  # Clean up any extra whitespace
        if question:  # Ensure the question is not empty
            # Append the cleaned question and answer to the list
            cleaned_data.append({'Question': question, 'Answer': answer})

# Convert the list to a DataFrame
cleaned_df = pd.DataFrame(cleaned_data)

# Save the cleaned dataset to a new file
cleaned_df.to_csv('./cleaned_dataset.csv', index=False)

# Reformat the dataset into the model's required format
with open('./formatted_output.txt', 'w', encoding='utf-8') as f:
    for index, row in cleaned_df.iterrows():
        formatted_text = f"<s>[INST] {row['Question']} [/INST] {row['Answer']} </s>\n"
        f.write(formatted_text)

In [5]:
model = AutoModelForCausalLM.from_pretrained(pretrained_model_name_or_path = "meta-llama/Llama-2-7b-chat-hf",
                                                   quantization_config = BitsAndBytesConfig(load_in_4bit = True, bnb_4bit_compute_dtype = getattr(torch, "float16"), bnb_4bit_quant_type = "nf4"))
model.config.use_cache = False
model.config.pretraining_tp = 1

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(pretrained_model_name_or_path = "meta-llama/Llama-2-7b-chat-hf", trust_remote_code = True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [7]:
# Load the dataset from the formatted text file
dataset = load_dataset('text', data_files='./formatted_output.txt')

training_arguments = TrainingArguments(output_dir = "./results", per_device_train_batch_size = 4, max_steps = 2500, save_steps=250, logging_steps=100, learning_rate=5e-5)

Generating train split: 0 examples [00:00, ? examples/s]

In [8]:
sft_trainer = SFTTrainer(
    model=model,
    args=training_arguments,
    train_dataset=dataset['train'],
    tokenizer=tokenizer,
    peft_config=LoraConfig(task_type="CAUSAL_LM", r=4, lora_alpha=16, lora_dropout=0.1),
    dataset_text_field="text"
)

/usr/local/lib/python3.10/dist-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1445 [00:00<?, ? examples/s]

In [9]:
sft_trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, *

Step,Training Loss
100,2.921200
200,1.805700
300,1.547700
400,1.383400
500,1.255300
600,1.187600
700,1.240600
800,1.030400
900,1.016800
1000,0.974400


/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an except

TrainOutput(global_step=2500, training_loss=1.026860041809082, metrics={'train_runtime': 3833.0094, 'train_samples_per_second': 2.609, 'train_steps_per_second': 0.652, 'total_flos': 1.77401026043904e+16, 'train_loss': 1.026860041809082, 'epoch': 6.91})

In [10]:
user_prompt = "What is AO code?"
text_generation_pipeline = pipeline(task = "text-generation", model = model, tokenizer = tokenizer, max_length = 300)
model_answer = text_generation_pipeline(f"<s>[INST] {user_prompt} [/INST]")
print(model_answer[0]['generated_text'])

Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/usr/local/lib/python3.10/dist-packages/torch/_dynamo/eval_frame.py:600: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<s>[INST] What is AO code? [/INST] Area Office (AO) code is a four digit code that identifies the office where an Assessing Officer (AO) is posted.  The AO code is used for the purpose of e-filing of returns, SFT, TDS/TCS returns, etc.  The AO code is available on the Assessing Officer (AO) Code page of the Income Tax website.  The AO code is also printed on the first page of the return forms.  For more details, please visit https://www.incometax.gov.in/iec/foportal/help/faqs/ao-code  to know the AO code for your office.  Please visit https://www.incometax.gov.in/iec/foportal/income-tax-software to download the return forms.  For more details, please visit https://www.incometax.gov.in/iec/foportal/help/faqs/ao-code to know the AO code for your office.  Please visit https://www.incometax.gov.in/iec/foportal/income-tax-software to download the return forms.  For more details, please visit https://www.incometax.gov.in/iec/foport


In [ ]:
text_generation_pipeline = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=250,
    do_sample=False,
)

def chatbot():
    print("Chatbot is running... Type 'exit' to stop.")
    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break
        else:
            # Format the input as required by the model
            formatted_input = f"<s>[INST] {user_input} [/INST] "
            # Generate the response
            response = text_generation_pipeline(formatted_input)
            # Print the generated response
            print("Chatbot:", response[0]['generated_text'].replace(formatted_input, "").strip())

# Run the chatbot
chatbot()

Chatbot is running... Type 'exit' to stop.
You: Can I surrender additional PAN through PAN change request application?
Chatbot: Yes, you may surrender your PAN through PAN change request application.  You may https://www.protean-tinpan.com/services/pan/pan-index.html  for more details.  You may submit the application with the supporting documents at https://www.protean-tinpan.com/services/pan/pan-application-online.html  to the respective ITD office.  You may https://www.protean-tinpan.com/services/pan/pan-faqs.html  to know the detailed procedure of PAN change request.  Please https://www.protean-tinpan.com/services/pan/pan-contact-us.html  to know the contact details of ITD.  Please https://www.protean-tinpan.com/services/pan/pan-helpdesk.html  to know the helpdesk contact details of ITD.  Please https://www.protean-tinpan.com/services/pan/pan-feedback.html
You: I have lost my PAN card. Please provide my PAN number to re-apply.
Chatbot: We request you to contact Income Tax Department